# (Q)分子構造+(R)理由+(A)物性データセットのLLMによる学習と予測
- Q&A: 融点データセットを使用
- R: 自分自身で考えさせて､正解のデータを学習させる

In [1]:
%reload_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from transformers import AutoTokenizer
import pandas as pd
import random
import copy
import glob
import json
from datetime import datetime
from llmchem.utils import mk_dir,clean_vram

#import clear_output

from IPython.display import clear_output

/home/setup/miniconda3/envs/chemllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#dataset settings
n_test=50 #number of testing data
n_train_check=50 #number of training data for checking (i.e., checking everything takes too long, so we check only a part of training data)
n_GPT_reasoning=10 # number of reasoning data made by GPT
n_generation_iterations=300   # trial numbers to generate new self reasoning data
max_generations=10**5

#model settings
model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
target_modules= [
    "lm_head",
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "gate",
    "w1",
    "w2",
    "w3"
]

model_name=f"meta-llama/Llama-2-7b-chat-hf"
target_modules= [
    #"embed_tokens",
    "lm_head",
    #"q_proj",
    #"k_proj",
    "v_proj",
    "o_proj",
    "gate_proj",
    "up_proj",
    #"down_proj",
]



#LoRA settings
r=32
lora_alpha=r
bit=16
#bit=8
#bit=4

#train settings
gradient_checkpointing =False
per_device_train_batch_size=1
epochs=3
lr=10**-5

#device settings
device_map="auto"

#dataset path
dataset_path="dataset/231225AutoReasoning/240117best_reason_record_3k.csv"

#project path
project_dir=f"results/projects/240118llama7b_{n_GPT_reasoning}"

#reasoning options
error_threshold=30  # if abolute error is smaller than this, add to training data

In [3]:
mk_dir(project_dir)
mk_dir(project_dir+"/eval")
mk_dir(project_dir+"/self_reasoning")
mk_dir(project_dir+"/train")

In [4]:
from llmchem.model import init_model
from llmchem.train import train_model
from llmchem.eval import eval_model
from llmchem.reasoning import self_reasoning
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [5]:
#load base dataset

df=pd.read_csv(dataset_path)
dataset=df.to_dict(orient="records")
random.seed(0)
random.shuffle(dataset)

base_train_dataset=dataset[:-n_test]
example_reasoning_dataset=base_train_dataset[:n_GPT_reasoning]
test_dataset=dataset[-n_test:]

In [6]:
#Loop: training, evaluation, data generation
for generation in range(max_generations):
    clear_output()
    print(f"Generation: {generation}")
    #prepare train dataset

    ## reason data made by GPT4
    train_dataset=copy.deepcopy(example_reasoning_dataset)

    print(f"GPT-generated reasons: {len(train_dataset)}")

    ## reason data made by model itself
    for path in glob.glob(f"{project_dir}/self_reasoning/*.json"):
        with open(path) as f:
            train_dataset.append(json.load(f))

    print(f"All-generated reasons: {len(train_dataset)}")
    random.shuffle(train_dataset)

    #train model
    clean_vram()
    model=init_model(model_name, r, lora_alpha, target_modules, bit=bit,device_map=device_map)
    train_result=train_model(model,tokenizer,train_dataset,
                    project_dir=project_dir,
                    epochs=epochs,
                    lr=lr,
                    per_device_train_batch_size=per_device_train_batch_size,
                    gradient_checkpointing=gradient_checkpointing,
                    )

    #eval

    train_check_dataset=copy.deepcopy(train_dataset[:n_train_check])
    random.shuffle(train_check_dataset)
    train_eval_result=eval_model(model,tokenizer,train_check_dataset,
                                f"{project_dir}/eval",
                                n_prompt_examples=3,
                                prompt_dataset=example_reasoning_dataset,
                                prefix=f"train_{generation}"
                                )

    test_eval_result=eval_model(model,tokenizer,test_dataset,
                                f"{project_dir}/eval",
                                n_prompt_examples=3,
                                prompt_dataset=example_reasoning_dataset,
                                prefix=f"test_{generation}"
                                )

    #generate additional training data by self-reasoning
    self_reasoning(model,tokenizer,base_train_dataset,
                example_reasoning_dataset,project_dir,
                generation=generation,
                n_iterations=n_generation_iterations,
                error_threshold=error_threshold,
                n_max_trials=2)

Generation: 0
GPT-generated reasons: 10
All-generated reasons: 10
Using fp16 mode


The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.
Map: 100%|██████████| 10/10 [00:00<00:00, 1092.64 examples/s]
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


  0%|          | 0/10 [00:00<?, ?it/s]

promlem 1 / 10


...

----


The compound has a complex structure with multiple functional groups that impact its melting point.
- Benzene ring: The base unit benzene has a melting point of 5.5 degrees Celsius.
- Hydrazine group: Hydrazones typically raise the melting point due to hydrogen bonding and increased polarity. A rough estimate for its contribution could be +50 degrees.
- Carbonyl group: Carbonyl groups also raise the melting point due to increased polarity and resonance stability. This can be estimated to contribute about +60 degrees.
- Methyl groups: Methyl groups increase the molecular weight leading to higher melting points, although the contribution is typically small. Estimated contribution might be +5 degrees.
- Cumulative impact: The individual contributions of each group can be summed up, but it's also necessary to consider that the presence of multiple polar and hydrogen bonding functional groups in one molecule can lead to synergistic effects, amplifying the increase in melting point be

 52%|█████▏    | 26/50 [09:17<10:56, 27.36s/it]

----


- The presence of a hydroxyl group increases the melting point due to increased hydrogen bonding. Estimated contribution of +50 degrees.
- The macrocyclic ring structure provides rigidity and potential for intermolecular hydrogen bonding, estimated to increase melting point by +40.
- The carboxylic acid functional group also contributes to the melting point due to increased polarity and hydrogen bonding, estimated at +30.
- The stereochemistry of the molecule may also contribute to the melting point, although the effect is less pronounced in this case. Estimated contribution of +10.
##Prediction: 130.0


#Problem
actual:  161.35 predicted:  130.0
promlem 27 / 50


 54%|█████▍    | 27/50 [09:49<11:01, 28.74s/it]

----


The molecule ethyl 2-amino-4-(3,4-dimethylphenyl)thiophene-3-carboxylate has a complex structure with multiple functional groups. The thiophene ring itself has a melting point of around -12°C, while the amino group adds a further -10°C. The carboxyl group is a polar functional group that can increase the melting point by around 20°C. The phenyl group attached to the thiophene ring through a methylene linkage adds a further +60°C due to its aromatic character. The total estimated impact on the melting point from all functional groups is: -12 (thiophene) + -10 (amino group) + 20 (carboxyl group) + 60 (phenyl group) = 82°C.
##Prediction: 82.0


#Problem
actual:  82.0 predicted:  82.0
promlem 28 / 50
